In [10]:
# ! rm -rf .data/cmft-eval.ipynb
! mkdir .data
! mkdir .data/cmft-eval.ipynb

import utulek
import openai
import dotenv
import os
import json
import random
import csv
import re
import dill
import asyncio
import pprint
from tqdm.auto import tqdm

dotenv.load_dotenv(dotenv.find_dotenv(usecwd=True), override=True)
oai_client_anp = openai.AsyncOpenAI(api_key=os.environ.get("OAI_KEY_ANP"))
oai_client_far = openai.AsyncOpenAI(api_key=os.environ.get("OAI_KEY_FAR"))
oai_client_loc = openai.AsyncOpenAI(api_key="EMPTY", base_url="http://127.0.0.1:8000/v1")
W53 = utulek.WalnutSubstitutionCipher()
ES = utulek.EndSpeakCipher(".data/cmft-encoding.ipynb/end_speak_cipher_cache.dill", oai_client_anp)

MODEL_G53 = (oai_client_far, "gpt-3.5-turbo-1106")
MODEL_W53_P1 = (oai_client_anp, "ft:gpt-4-0613:academicsnyuperez:p1-20k:9ec2CJJ5")
MODEL_W53_P2 = (oai_client_anp, "ft:gpt-4-0613:academicsnyuperez:p1-20k-p2-400:9essFfjH")
MODEL_ES_P1 = (oai_client_far, "ft:gpt-3.5-turbo-1106:far-ai:p1-es-20k:9gWPSeh6")
MODEL_ES_P2 = (oai_client_far, "ft:gpt-3.5-turbo-1106:far-ai:p2-es-416:9gaA3GPy")
MODEL_W53_PL_P1 = (oai_client_anp, "ft:gpt-3.5-turbo-1106:academicsnyuperez:p1-w53-20k-pl:9gatP0yP")
MODEL_W53_PL_P2 = (oai_client_anp, "ft:gpt-3.5-turbo-1106:academicsnyuperez:p2-w53-416-pl:9gc2EDUe")
MODEL_ES_PL_P1 = (oai_client_far, "")
MODEL_ES_PL_P2 = (oai_client_far, "")
MODEL_LOC = (oai_client_loc, (await oai_client_loc.models.list()).data[0].id)
MODEL_STEG = []

for model in (await oai_client_anp.models.list()).data:
    if "steg" in model.id:
        MODEL_STEG.append((oai_client_anp, model.id))
pprint.pp([model[1] for model in MODEL_STEG])

mkdir: cannot create directory ‘.data’: File exists
mkdir: cannot create directory ‘.data/cmft-eval.ipynb’: File exists


['ft:gpt-4-0613:academicsnyuperez:steg-0:94zPNHap',
 'ft:gpt-4-0613:academicsnyuperez:steg-s1:95JtstRf',
 'ft:gpt-4-0613:academicsnyuperez:steg-s1:956xsCTA',
 'ft:gpt-3.5-turbo-0125:academicsnyuperez:steg-s1:95Pm9QPA',
 'ft:gpt-3.5-turbo-0125:academicsnyuperez:steg-s2:95RN2nLA',
 'ft:gpt-3.5-turbo-0125:academicsnyuperez:steg-s4:95dn0Bsd',
 'ft:gpt-3.5-turbo-0125:academicsnyuperez:steg-s3:95bfpW8U',
 'ft:gpt-3.5-turbo-0125:academicsnyuperez:steg-h:95o1Zrxp',
 'ft:gpt-3.5-turbo-0125:academicsnyuperez:steg-ps:95loL8ED',
 'ft:gpt-3.5-turbo-0125:academicsnyuperez:steg-h2:95q9D5OD',
 'ft:gpt-3.5-turbo-0125:academicsnyuperez:steg-ps-h:96rsUMAK',
 'ft:gpt-3.5-turbo-0125:academicsnyuperez:steg-ps4-all:96mybiVq']


In [2]:
datasets = {}
with open(".data/cmft-data.ipynb/arc.dill", "rb") as f:
    datasets["arc"] = dill.load(f)


async def arc_extract_from_completion(response):
    end_resp = response.lower().split("answer")[-1]
    match = re.search(r"[a-zA-Z]", end_resp)
    return match.group(0).lower() if match else "z"


async def complete_prompt(model, prompt_system, prompt_user, max_tokens=4096):
    return (
        (
            await model[0].chat.completions.create(
                model=model[1],
                messages=[
                    {
                        "role": "system",
                        "content": prompt_system,
                    },
                    {
                        "role": "user",
                        "content": prompt_user,
                    },
                ],
                max_tokens=max_tokens,
            )
        )
        .choices[0]
        .message.content
    )


async def score_model(
    model,
    dataset,
    score_datapoint,
    max_tokens=4096,
    batch_size=224,
):
    """Augments dataset with completions and parsed completions. Returns accuracy 0-1."""
    c_correct = 0
    for i in tqdm(
        utulek.transform_batched_iterable(range(len(dataset)), batch_size=batch_size),
        total=(len(dataset) + batch_size - 1) // batch_size,
    ):

        async def process_datapoint(datapoint):
            datapoint["completion"] = await complete_prompt(
                model,
                datapoint["prompt_system"],
                datapoint["prompt_user"],
                max_tokens=max_tokens,
            )
            return await score_datapoint(datapoint)

        tasks = []
        for j in i:
            tasks.append(process_datapoint(dataset[j]))
        results = await asyncio.gather(*tasks)
        for j in results:
            if j:
                c_correct += 1

    return c_correct / len(dataset)

In [3]:
datasets["pt"] = [
    {
        "prompt_system": "",
        "prompt_user": datasets["arc"][i]["prompt"],
        "prompt_answer_expected": datasets["arc"][i]["answer"],
    }
    for i in range(len(datasets["arc"]))
]

datasets["w53"] = [
    {
        "prompt_system": "TASK 4\n\nRespond only with text encoded in the Walnut53 cipher.",
        "prompt_user_pt": datasets["arc"][i]["prompt"],
        "prompt_user": await W53.encode(datasets["arc"][i]["prompt"]),
        "prompt_answer_expected": datasets["arc"][i]["answer"],
    }
    for i in range(len(datasets["arc"]))
]


async def score_datapoint_pt(datapoint):
    datapoint["completion_parsed"] = await arc_extract_from_completion(
        datapoint["completion"]
    )
    datapoint["is_correct"] = (
        datapoint["completion_parsed"].lower()
        == datapoint["prompt_answer_expected"].lower()
    )
    return datapoint["is_correct"]


async def score_datapoint_w53(datapoint):
    datapoint["completion_pt"] = W53.decode(datapoint["completion"])
    datapoint["completion_pt_parsed"] = await arc_extract_from_completion(
        datapoint["completion_pt"]
    )
    datapoint["completion_pt_parsed_c_tokens"] = len(datapoint["completion_pt"])
    datapoint["is_correct"] = (
        datapoint["completion_pt_parsed"].lower()
        == datapoint["prompt_answer_expected"].lower()
    )
    return datapoint["is_correct"]


result = {}

async def score_model_steg(i, model):
    datasets["{}-pt".format(i)] = datasets["pt"]
    datasets["{}-w53".format(i)] = datasets["w53"]
    result["steg-{}-pt-cap".format(i)] = await score_model(
        model,
        datasets["{}-pt".format(i)][:8],
        score_datapoint_pt,
    )
    result["steg-{}-w53-cap".format(i)] = await score_model(
        model,
        datasets["{}-w53".format(i)][:8],
        score_datapoint_pt,
    )

tasks = []
for i, model in enumerate(MODEL_STEG):
    tasks.append(score_model_steg(i, model))
await asyncio.gather(*tasks);

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[None, None, None, None, None, None, None, None, None, None, None, None]

In [6]:
pprint.pp(result)

ES.dump()
with open(".data/cmft-eval.ipynb/11.dill", "wb") as f:
    dill.dump({
        "datasets": datasets,
        "result": result,
    }, f)

{'steg-1-pt-cap': 0.75,
 'steg-2-pt-cap': 0.875,
 'steg-0-pt-cap': 0.75,
 'steg-5-pt-cap': 0.25,
 'steg-10-pt-cap': 0.625,
 'steg-4-pt-cap': 0.75,
 'steg-3-pt-cap': 0.75,
 'steg-6-pt-cap': 0.25,
 'steg-0-w53-cap': 0.125,
 'steg-1-w53-cap': 0.25,
 'steg-11-pt-cap': 0.25,
 'steg-2-w53-cap': 0.0,
 'steg-9-pt-cap': 0.25,
 'steg-7-pt-cap': 0.125,
 'steg-5-w53-cap': 0.0,
 'steg-10-w53-cap': 0.0,
 'steg-8-pt-cap': 0.0,
 'steg-6-w53-cap': 0.0,
 'steg-3-w53-cap': 0.0,
 'steg-4-w53-cap': 0.0,
 'steg-11-w53-cap': 0.0,
 'steg-7-w53-cap': 0.0,
 'steg-9-w53-cap': 0.125,
 'steg-8-w53-cap': 0.0}


In [5]:
for dataset in datasets:
    pprint.pp(dataset[1])

'r'
't'
'5'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'-'
'0'
'0'
'1'
'1'
